# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
#file_path = r"C:\Users\glamh\Downloads\cities.csv"
city_data_df = pd.read_csv(r"C:\Users\glamh\Downloads\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,281.28,91,100,17.17,TF,['dt']
1,1,kristiansund,63.1115,7.7320,277.51,87,100,6.17,NO,['dt']
2,2,codrington,-38.2667,141.9667,290.98,59,87,8.73,AU,['dt']
3,3,invercargill,-46.4000,168.3500,288.16,83,100,2.68,NZ,['dt']
4,4,sinazongwe,-17.2614,27.4618,297.02,87,100,2.02,ZM,['dt']


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
!pip install cartopy geoviews

  Obtaining dependency information for cartopy from https://files.pythonhosted.org/packages/e1/a2/a450fe2d20be42c6666a5146697a6ea1ab9caa1940cce892af02b492c3db/Cartopy-0.23.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for geoviews from https://files.pythonhosted.org/packages/5e/1f/62ccb9e157acd3c133179a0c77ba68e3afb4cce321e052eb0733b786afe0/geoviews-1.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyshp>=2.3 from https://files.pythonhosted.org/packages/98/2f/68116db5b36b895c0450e3072b8cb6c2fac0359279b182ea97014d3c8ac0/pyshp-2.3.1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ------------------------------------ --- 51.2/56.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 56.0/56.0 kB 737.6 kB/s eta 0:00:00
  Obtaining dependency information for bokeh<3.5.0,>=3.4.0 from https://files.pythonhosted.org/packages/1a/02/b312d65b83d0ec17d73e1594118e1163a4d79546925

In [37]:
%%capture --no-display

import holoviews as hv  # Import the holoviews library

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\glamh\\anaconda3\\Lib\\site-packages\\panel\\assets\\arc_spinner.svg'

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_Cities = city_data_df[city_data_df["Max Temp"] > 23]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
Ideal_Cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,281.28,91,100,17.17,TF,['dt']
1,1,kristiansund,63.1115,7.7320,277.51,87,100,6.17,NO,['dt']
2,2,codrington,-38.2667,141.9667,290.98,59,87,8.73,AU,['dt']
3,3,invercargill,-46.4000,168.3500,288.16,83,100,2.68,NZ,['dt']
4,4,sinazongwe,-17.2614,27.4618,297.02,87,100,2.02,ZM,['dt']


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
#Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

#Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

#Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,91,
1,kristiansund,NO,63.1115,7.7320,87,
2,codrington,AU,-38.2667,141.9667,59,
3,invercargill,NZ,-46.4000,168.3500,83,
4,sinazongwe,ZM,-17.2614,27.4618,87,
...,...,...,...,...,...,...
592,luba,GQ,3.4568,8.5547,78,
593,akrehamn,NO,59.2667,5.1833,87,
594,nadym,RU,65.5333,72.5167,100,
595,omarska,BA,44.8900,16.8992,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
port-aux-francais - nearest hotel: Keravel
kristiansund - nearest hotel: Comfort Hotel Fosna
codrington - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
sinazongwe - nearest hotel: Lake View Lodge
vilyuchinsk - nearest hotel: Победа
waitangi - nearest hotel: Hotel Chathams
mitsamiouli - nearest hotel: Hotel le Paradis
ribeira grande - nearest hotel: Hospedaria JSF
port mathurin - nearest hotel: Escale Vacances
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
umea - nearest hotel: Hotell Dragonen
green river - nearest hotel: Hampton Inn & Suites
grytviken - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
zhaqsy - nearest hotel: Гостиница-кафе 24/7
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
turpan - nearest hotel: Oriental
blackmans bay - nearest hotel: Villa Howden
constantia - nearest hotel: Hotel Dali
villa de arista - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Co

zabaykal'sk - nearest hotel: Гостиница Русь
nantucket - nearest hotel: No hotel found
dunedin - nearest hotel: Scenic Hotel Dunedin City
aasiaat - nearest hotel: SØMA
el granada - nearest hotel: Beach House
twon-brass - nearest hotel: KC Hotel
sinop - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
santona - nearest hotel: Hotel Juan de la Cosa
sosnytsya - nearest hotel: Убідь
vodnyy - nearest hotel: No hotel found
maun - nearest hotel: Center Lodge Conference Center
atafu village - nearest hotel: No hotel found
cullinan - nearest hotel: No hotel found
bakanas - nearest hotel: Карой
howrah - nearest hotel: Sun India Guest House
stanley - nearest hotel: Hotel 52
arraial do cabo - nearest hotel: No hotel found
beaupreau - nearest hotel: Le Logis d'Elbée
vilhena - nearest hotel: No hotel found
tutoia - nearest hotel: Pousada San Vicente
aoulef - nearest hotel: دار الضياف
pudozh - nearest hotel: Карел-Онего
east ballina - nearest hotel: Ballina Bea

vetlanda - nearest hotel: No hotel found
hasaki - nearest hotel: Choshi Plaza Hotel
wanning - nearest hotel: No hotel found
peniche - nearest hotel: Residencial Maciel
inirida - nearest hotel: Residencias El Turista del Guainia
pervomayskiy - nearest hotel: No hotel found
edmundston - nearest hotel: Comfort Inn
boguchany - nearest hotel: Полет
ketchikan - nearest hotel: Cape Fox Lodge
peace river - nearest hotel: New Ridge Inn
kiunga - nearest hotel: No hotel found
chinde - nearest hotel: Kanyum cottage village
loreto - nearest hotel: No hotel found
berdigestyakh - nearest hotel: No hotel found
korsakov - nearest hotel: гостиница "Альфа"
bullsbrook - nearest hotel: No hotel found
nuku'alofa - nearest hotel: City Hotel
san-pedro - nearest hotel: Hôtel La Cité De San Pédro
eilat - nearest hotel: Melony Club
cam ranh - nearest hotel: Khách sạn Thái Bình
oliveira dos brejinhos - nearest hotel: No hotel found
yacuiba - nearest hotel: Residencial Balcarce
amga - nearest hotel: "АммА"
miracem

temerluh - nearest hotel: Hotel Seri Malaysia
correntina - nearest hotel: Pousada Silva
san julian - nearest hotel: No hotel found
divnomorskoye - nearest hotel: панс. Приморский
tabuk - nearest hotel: Davidson Hotel
manaure - nearest hotel: hotel palaima
shimoda - nearest hotel: ホテルウラガ
menaka - nearest hotel: No hotel found
villa ojo de agua - nearest hotel: No hotel found
amungan - nearest hotel: Tanyaw Beach Resort
diekirch - nearest hotel: Hôtel du Parc
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
abadan - nearest hotel: خانه معلم
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
nishinoomote - nearest hotel: ゼウスハウス
pariaman - nearest hotel: HOTEL NAN TONGGA
laredo - nearest hotel: Rialto Hotel
luba - nearest hotel: Hotel Jones
akrehamn - nearest hotel: Park Inn Haugesund Airport
nadym - nearest hotel: Янтарная
omarska - nearest hotel: Bomar
huanggang - nearest hotel: 聚贤园迎宾馆


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,91,Keravel
1,kristiansund,NO,63.1115,7.7320,87,Comfort Hotel Fosna
2,codrington,AU,-38.2667,141.9667,59,No hotel found
3,invercargill,NZ,-46.4000,168.3500,83,The Grand
4,sinazongwe,ZM,-17.2614,27.4618,87,Lake View Lodge
...,...,...,...,...,...,...
592,luba,GQ,3.4568,8.5547,78,Hotel Jones
593,akrehamn,NO,59.2667,5.1833,87,Park Inn Haugesund Airport
594,nadym,RU,65.5333,72.5167,100,Янтарная
595,omarska,BA,44.8900,16.8992,90,Bomar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\glamh\\anaconda3\\Lib\\site-packages\\panel\\assets\\arc_spinner.svg'

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)